In [284]:
library(vegan)
library(tidyverse)
library(pairwiseAdonis)
library(emmeans)

In [285]:
dados <- read.csv('../dados/timbre_descritores.csv')

### Auxiliar

In [286]:
descritores <- c("centroide", "atk_time")
fatores <- c("material", "nota", "metodo")

In [287]:
printCabecalho <- function(texto, largura = 70) {
  cat("\n", rep("=", largura), "\n", sep = "")
  cat(texto, "\n")
  cat(rep("=", largura), "\n\n", sep = "")
}

In [288]:
prepDados <- function(df, descritores) {
  dados_padronizados <- df %>%
    select(all_of(descritores)) %>%
    scale() %>%
    as.data.frame()
  
  return(dados_padronizados)
}

# PERMANOVA

In [290]:
calcMatrizDistancia <- function(dados, metodo = "euclidean") {
  if (is.data.frame(dados) && ncol(dados) > 1) {
    return(vegdist(dados, method = metodo))
  }
  return(dist(dados))
}

In [291]:
execPermanova <- function(matriz_dist, dados, fatores, permutations, by) {
  termos <- paste(fatores, collapse = " * ")
  formula <- as.formula(paste("matriz_dist ~", termos))
  
  modelo <- adonis2(
    formula,
    data = dados,
    permutations = permutations,
    by = by
  )
  
  return(modelo)
}

In [292]:
analiseUnivariada <- function(df, descritor, fatores, permutations, by) {
  printCabecalho(paste(toupper(descritor)))
  
  dados <- data.frame(
    descritor_valor = scale(df[[descritor]]),
    df[fatores]
  )

  matriz_dist <- calcMatrizDistancia(dados$descritor_valor)
  modelo <- execPermanova(matriz_dist, dados, fatores, permutations, by = by)
  
  print(modelo)
}


In [293]:
analiseMultivariada <- function(df, descritores, fatores, permutations, by) {
  titulo <- paste(paste(toupper(descritores), collapse = " + "))
  printCabecalho(titulo)
  
  dados_combinados <- prepDados(df, descritores)
  matriz_dist <- calcMatrizDistancia(dados_combinados, "euclidean")
  modelo <- execPermanova(matriz_dist, df, fatores, permutations, by = by)
  
  print(modelo)
}


In [294]:
PERMANOVA <- function(df, descritores, fatores, permutations = 9999, by) {

  
  # Análises individuais
  for (descritor in descritores) {
    modelo <- analiseUnivariada(df, descritor, fatores, permutations, by = by)
  }
  
  # Análise combinada
  modelo_combinado <- analiseMultivariada(df, descritores, fatores, permutations, by = by)
}


## Univariada e multivariada

In [295]:
PERMANOVA(
  df = dados,
  descritores = descritores,
  fatores = fatores,
  permutations = 9999,
  by = 'term'
)


CENTROIDE 

Permutation test for adonis under reduced model
Terms added sequentially (first to last)
Permutation: free
Number of permutations: 9999

adonis2(formula = formula, data = dados, permutations = permutations, by = by)
                     Df SumOfSqs      R2          F Pr(>F)    
material              1    0.012 0.00015     1.7779 0.1834    
nota                  3    1.797 0.02275    90.1918 0.0001 ***
metodo                1   71.196 0.90121 10719.6081 0.0001 ***
material:nota         3    0.065 0.00082     3.2629 0.0307 *  
material:metodo       1    0.044 0.00056     6.6722 0.0129 *  
nota:metodo           3    5.063 0.06409   254.0963 0.0001 ***
material:nota:metodo  3    0.398 0.00504    19.9707 0.0001 ***
Residual             64    0.425 0.00538                      
Total                79   79.000 1.00000                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

ATK_TIME 

Permutation test for adonis under reduced model
Terms added se

# Global

In [296]:
PERMANOVA(
  df = dados,
  descritores = descritores,
  fatores = fatores,
  permutations = 9999,
  by = NULL
)


CENTROIDE 

Permutation test for adonis under reduced model
Permutation: free
Number of permutations: 9999

adonis2(formula = formula, data = dados, permutations = permutations, by = by)
         Df SumOfSqs      R2      F Pr(>F)    
Model    15   78.575 0.99462 788.71  1e-04 ***
Residual 64    0.425 0.00538                  
Total    79   79.000 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

ATK_TIME 

Permutation test for adonis under reduced model
Permutation: free
Number of permutations: 9999

adonis2(formula = formula, data = dados, permutations = permutations, by = by)
         Df SumOfSqs      R2      F Pr(>F)    
Model    15   66.532 0.84217 22.767  1e-04 ***
Residual 64   12.468 0.15783                  
Total    79   79.000 1.00000                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

CENTROIDE + ATK_TIME 

Permutation test for adonis under reduced model
Permutation: free
Number of permutations: 9

# PERMDISP

In [310]:
PERMDISP <- function(df, descritores, fator, permutations = 9999) {
  
  dados <- prepDados(df, descritores)
  matriz_dist <- calcMatrizDistancia(dados, "euclidean")
  
  bd <- betadisper(matriz_dist, group = df[[fator]])
  
  # Testes
  printCabecalho(paste(toupper('ANOVA PARAMÉTRICA')))
  anova_disp <- anova(bd)
  print(anova_disp)

  printCabecalho(paste(toupper('TESTE POR PERMUTAÇÃO')))
  perm_disp <- permutest(bd, permutations = permutations)
  print(perm_disp)

  printCabecalho(paste(toupper('TUKEY HSD')))
  tuk_disp <- TukeyHSD(bd)
  print(tuk_disp)
}


In [311]:
PERMDISP(
  df = dados,
  descritores = descritores,
  fator = "metodo",
  permutations = 9999
)


ANOVA PARAMÉTRICA 

Analysis of Variance Table

Response: Distances
          Df Sum Sq Mean Sq F value    Pr(>F)    
Groups     1 3.5440  3.5440  29.437 6.295e-07 ***
Residuals 78 9.3906  0.1204                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

TESTE POR PERMUTAÇÃO 


Permutation test for homogeneity of multivariate dispersions
Permutation: free
Number of permutations: 9999

Response: Distances
          Df Sum Sq Mean Sq      F N.Perm Pr(>F)    
Groups     1 3.5440  3.5440 29.437   9999  1e-04 ***
Residuals 78 9.3906  0.1204                         
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

TUKEY HSD 

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = distances ~ group, data = df)

$group
                     diff      lwr       upr p adj
natural-bending 0.4209494 0.266487 0.5754119 6e-07



In [299]:
PERMDISP(
  df = dados,
  descritores = descritores,
  fator = "material",
  permutations = 9999
)


ANOVA PARAMÉTRICA 

Analysis of Variance Table

Response: Distances
          Df  Sum Sq Mean Sq F value Pr(>F)  
Groups     1  1.2433 1.24334  3.4454 0.0672 .
Residuals 78 28.1478 0.36087                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

TESTE POR PERMUTAÇÃO 


Permutation test for homogeneity of multivariate dispersions
Permutation: free
Number of permutations: 9999

Response: Distances
          Df  Sum Sq Mean Sq      F N.Perm Pr(>F)  
Groups     1  1.2433 1.24334 3.4454   9999 0.0658 .
Residuals 78 28.1478 0.36087                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

TUKEY HSD 

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = distances ~ group, data = df)

$group
                       diff        lwr        upr     p adj
plastico-madeira -0.2493332 -0.5167558 0.01808934 0.0672045



In [300]:
PERMDISP(
  df = dados,
  descritores = descritores,
  fator = "nota",
  permutations = 9999
)


ANOVA PARAMÉTRICA 

Analysis of Variance Table

Response: Distances
          Df Sum Sq Mean Sq F value Pr(>F)
Groups     3  4.195 1.39830  2.1269 0.1038
Residuals 76 49.965 0.65744               

TESTE POR PERMUTAÇÃO 


Permutation test for homogeneity of multivariate dispersions
Permutation: free
Number of permutations: 9999

Response: Distances
          Df Sum Sq Mean Sq      F N.Perm Pr(>F)
Groups     3  4.195 1.39830 2.1269   9999 0.1082
Residuals 76 49.965 0.65744                     

TUKEY HSD 

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = distances ~ group, data = df)

$group
             diff        lwr       upr     p adj
E4-A4  0.29112305 -0.3824013 0.9646474 0.6689337
F4-A4 -0.05923039 -0.7327548 0.6142940 0.9956208
G4-A4  0.50843274 -0.1650916 1.1819571 0.2034920
F4-E4 -0.35035344 -1.0238778 0.3231709 0.5241389
G4-E4  0.21730969 -0.4562147 0.8908341 0.8315242
G4-F4  0.56766313 -0.1058612 1.2411875 0.1287044



### Auxiliar

In [ ]:
colSignificancia <- function(resultados, coluna_p = "p_ajustado") {
  resultados$Significancia <- cut(
    resultados[[coluna_p]],
    breaks = c(0, 0.001, 0.01, 0.05, 1),
    labels = c("***", "**", "*", "n.s."),
    include.lowest = TRUE
  )
  resultados[order(resultados[[coluna_p]]), ]
}

# Post-hoc principal: PERMANOVA par-a-par

In [301]:
execPermanovaPar <- function(matriz_dist, df, fator, par, permutations) {
  indices <- df[[fator]] %in% par
  D_par <- as.dist(as.matrix(matriz_dist)[indices, indices])
  df_par <- df[indices, ]
  
  formula_str <- paste("D_par ~", fator)
  
  resultado <- adonis2(as.formula(formula_str), data = df_par, 
                       permutations = permutations, by = NULL)
  
  data.frame(
    Par = paste(par[1], "vs", par[2]),
    F = resultado$F[1],
    R2 = resultado$R2[1],
    p = resultado$`Pr(>F)`[1]
  )
}


In [302]:
posthocPermanova <- function(df, descritores, fator, permutations = 9999) {
  
  printCabecalho(paste("POST-HOC PAR-A-PAR:", toupper(fator)))
  
  dados_padronizados <- prepDados(df, descritores)
  matriz_dist <- calcMatrizDistancia(dados_padronizados, "euclidean")
  
  combinacoes <- combn(unique(df[[fator]]), 2, simplify = FALSE)
  resultados <- do.call(rbind, lapply(combinacoes, function(par) {
    execPermanovaPar(matriz_dist, df, fator, par, permutations)
  }))
  
  resultados$p_ajustado <- p.adjust(resultados$p, method = "holm")
  resultados <- colSignificancia(resultados, "p_ajustado")
  
  print(resultados)
}

In [303]:
posthocPermanova(
  df = dados,
  descritores = descritores,
  fator = "metodo",
  permutations = 9999
)


POST-HOC PAR-A-PAR: METODO 

                 Par        F        R2     p p_ajustado Significancia
1 natural vs bending 324.0182 0.8059789 1e-04      1e-04           ***


In [304]:
posthocPermanova(
  df = dados,
  descritores = descritores,
  fator = "material",
  permutations = 9999
)


POST-HOC PAR-A-PAR: MATERIAL 

                  Par        F         R2      p p_ajustado Significancia
1 madeira vs plastico 0.808176 0.01025498 0.3771     0.3771          n.s.


In [305]:
posthocPermanova(
  df = dados,
  descritores = descritores,
  fator = "nota",
  permutations = 9999
)


POST-HOC PAR-A-PAR: NOTA 

       Par         F          R2      p p_ajustado Significancia
1 A4 vs F4 1.4225383 0.036084391 0.2279          1          n.s.
2 A4 vs G4 0.6280103 0.016257899 0.4475          1          n.s.
3 A4 vs E4 0.1507094 0.003950369 0.7924          1          n.s.
4 F4 vs G4 1.1032780 0.028214463 0.2926          1          n.s.
5 F4 vs E4 0.8406664 0.021643973 0.3720          1          n.s.
6 G4 vs E4 0.1682130 0.004407150 0.7472          1          n.s.


# Pós-hoc complementar em dispersão (se PERMDISP significativa)

In [306]:
posthocPermdisp <- function(df, descritores, fator, permutations = 9999) {
  
  printCabecalho(paste("POST-HOC DISPERSÃO:", toupper(fator)))
  
  dados_padronizados <- prepDados(df, descritores)
  matriz_dist <- calcMatrizDistancia(dados_padronizados, "euclidean")
  
  bd <- betadisper(matriz_dist, group = df[[fator]])
  
  # Tukey HSD
  distancias <- data.frame(grupo = df[[fator]], distancia = bd$distances)
  tukey_result <- TukeyHSD(aov(distancia ~ grupo, data = distancias))
  tukey_df <- as.data.frame(tukey_result$grupo)
  
  tabela <- data.frame(
    Par = rownames(tukey_df),
    Δdispersão = tukey_df$diff,
    IC_inf = tukey_df$lwr,
    IC_sup = tukey_df$upr,
    p_ajustado = tukey_df$`p adj`
  )
  
  tabela <- colSignificancia(tabela, "p_ajustado")
  
  print(tabela)
}

In [307]:
posthocPermdisp(
  df = dados,
  descritores = descritores,
  fator = "metodo",
  permutations = 9999
)


POST-HOC DISPERSÃO: METODO 

              Par Δdispersão   IC_inf    IC_sup   p_ajustado Significancia
1 natural-bending  0.4209494 0.266487 0.5754119 6.294209e-07           ***
